[View in Colaboratory](https://colab.research.google.com/github/central-ldn-data-sci/advent_of_code/blob/master/2018/python/2018_solutions.ipynb)

## Advent of Code 2018

Festive greetings. Last meetup we looked at the [advent of code 2017](https://adventofcode.com/2017). Advent of Code is a series of small programming puzzles for a variety of skill sets and skill levels in any programming language you like. People use them as a speed contest, interview prep, company training, university coursework, practice problems, or to challenge each other.

There is a new challenge posted each day between 1st - 25th December. These challenges are very accessible to begin with and then they get a bit harder each day. 

To help keep the festive spirit we hope to post solutions to this notebook for each of the 2018 challenges. So check back each day for solutions. 


---

## The challenges

For each challenge you will be given an input file. The input file that you are generated by the Advent of Code is specific to your user account. As a result the answers generated below will likely not be correct for your Advent of Code account - so no cheating and just copying these answers!

The following bsah script will download the files from the github to where you have forked/collab'd this notebook too. 

In [ ]:
%%bash
for i in {1..25};
do
  url=https://raw.githubusercontent.com/central-ldn-data-sci/advent_of_code/master/2018/python/input$i.txt
  if [[ `wget -S --spider $url 2>&1 | grep 'HTTP/1.1 200 OK'` ]];
  then 
    wget $url; 
  fi
done

In [80]:
# packages
import numpy as np
import pandas as pd
import re

### Day 1:

In [82]:
# %%writefile oj-day1.py
import numpy as np

data = np.loadtxt('input1.txt')
print(data.sum())

s = set([0]) # starts with freq. 0
m = 0
s2 = set(data.cumsum() + m)

while(s.isdisjoint(s2)): # returns TRUE if the set has no elements in common with s2
    s.update(data.cumsum() + m)
    m = data.sum() + m
    s2 = set(data.cumsum() + m)
print(tuple(s.intersection(s2))[0])

569.0
77696.0


### Day 2:

In [83]:
# %%writefile oj-day2.py
import numpy as np
from collections import Counter # Counter gives a table of the occurrences in the object passed.

data = np.loadtxt('input2.txt', dtype = "str")

two_sum = 0
three_sum = 0

for d in data:
    if (2 in Counter(d).values()): two_sum += 1 # for numpy strings counter tables the letters
    if (3 in Counter(d).values()): three_sum += 1

print(two_sum * three_sum)

def hamming(str1, str2):
    assert len(str1) == len(str2)
    return sum(ch1 != ch2 for ch1,ch2 in zip(str1,str2))

def common_lets(data):
    l = len(data)
    for i, d in enumerate(data):
        iter = 1
        while(i + iter < l):
            if(hamming(d, data[iter]) == 1):
                return("".join(ch1 for ch1,ch2 in zip(d,data[iter]) if ch1==ch2))
            iter += 1

print(common_lets(data))

5976
xretqmmonskvzupalfiwhcfdb


### Day 3:

In [84]:
# %%writefile oj-day3.py
import codecs
import numpy as np
import re

data = codecs.open('input3.txt', 'r', encoding='utf-8').readlines()
print(data[0])

# check max grid size
max_x = 0
max_y = 0
for d in data:
    y = int(re.match(r'#\d+\s@\s(.*),', d)[1])
    y2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s(.*)x', d)[1])
    max_y = y+y2 if y+y2 > max_y else max_y
    x = int(re.match(r'#\d+\s@\s\d+,(.*):', d)[1])
    x2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s\d+x(.*)$', d)[1])
    max_x = x+x2 if x+x2 > max_x else max_x


# make numpy grid
grid = np.zeros((max_x+1, max_y+1))
for i, d in enumerate(data):
    y = int(re.match(r'#\d+\s@\s(.*),', d)[1])
    y2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s(.*)x', d)[1])
    x = int(re.match(r'#\d+\s@\s\d+,(.*):', d)[1])
    x2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s\d+x(.*)$', d)[1])
    grid[x:x+x2, y:y+y2] += 1;
print(np.sum(grid > 1))
    
# check for completeness
for i, d in enumerate(data):
    y = int(re.match(r'#\d+\s@\s(.*),', d)[1])
    y2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s(.*)x', d)[1])
    x = int(re.match(r'#\d+\s@\s\d+,(.*):', d)[1])
    x2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s\d+x(.*)$', d)[1])
    if np.all(grid[x:x+x2, y:y+y2] == 1):
        print(i)
        break
  

#1 @ 604,670: 22x16

110389
551


### Day 4:

In [85]:
# %%writefile oj-day4.py
import codecs
import numpy as np
import re

data = codecs.open('input4.txt', 'r', encoding='utf-8').readlines()

# grab our times
times = []
for time in data:
    times.append([i for i in re.findall("[\d]+|wakes up|falls asleep", time)])
    
# sort our times
def abs_time(elem):
    return elem[0]*365*24*60 + elem[1]*30*24*60 + elem[2]*24*60 + elem[3]*60 + elem[4]

times.sort(key = abs_time) 

# create our dict of guard snoozes
snoozes = {}
for i, time in enumerate(times):
    year, month, day, hour, minute, action = time
    if action.isdigit():
        if action not in snoozes:
            snoozes[action] = np.zeros(60)
        guard = action
    if action == 'falls asleep':
        snoozes[guard][int(minute):int(times[i+1][4])] += 1

# then for part one we want to find the snooziest guard
sorted_snoozes = sorted(snoozes.items(), key = lambda e: np.sum(e[1]), reverse=True)
print(int(sorted_snoozes[0][0]) * sorted_snoozes[0][1].argmax())

# and for part two we want to find the guard with the most snoozy minute
sorted_snoozes = sorted(snoozes.items(), key = lambda e: np.max(e[1]), reverse=True)
print(int(sorted_snoozes[0][0]) * sorted_snoozes[0][1].argmax())

142515
5370


### Day 5

In [87]:
# %%writefile oj-day5_quicker.py
import codecs

def invert_case(char):
    return (char.lower() if char.isupper() else char.upper())

def remove_values_from_list(l, val):
    return [value for value in l if value not in val]

data = codecs.open('input5.txt', 'r', encoding='utf-8').readlines()
#data = list("dabAcCaCBAcCcaDA")

def remove_copies(data):
    cpy = data[:] # we don't need this in a function, but would outside as python does a shallow copy
    while("!" in cpy or len(cpy)==len(data)):
        cpy = remove_values_from_list(cpy, "!")
        invert = [invert_case(c) for c in cpy]
        for i in range(len(cpy)-1):
            if (cpy[i] != '!') and (cpy[i] == invert[i+1]):
                cpy[i:(i+2)] = '!'*2 # python will not iterate and replace each with !, we have to create that
    return(cpy)

print(len(remove_copies(list(data[0]))))  

lowers = set(str(data[0]).lower())
print(min([len(remove_copies(remove_values_from_list(data[0], {l, invert_case(l)}))) for l in lowers]))

10250
6188


### Day 6

In [88]:
# %%writefile oj-day6.py
from collections import defaultdict

def part1(lines):
    coords = set()
    max_x = max_y = 0

    for line in lines:
        x, y = map(int, line.split(", "))
        coords.add((x, y))
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    coord_id_to_point = {coord_id: point for coord_id, point in enumerate(coords, start=1)}
    region_sizes = defaultdict(int)
    infinite_ids = set()

    for i in range(max_x + 1):
        for j in range(max_y + 1):
            min_dists = sorted([(abs(x - i) + abs(y - j), coord_id) for coord_id, (x, y) in coord_id_to_point.items()])

            if len(min_dists) == 1 or min_dists[0][0] != min_dists[1][0]:
                coord_id = min_dists[0][1]
                region_sizes[coord_id] += 1

                if i == 0 or i == max_x or j == 0 or j == max_y:
                    infinite_ids.add(coord_id)

    return max(size for coord_id, size in region_sizes.items() if coord_id not in infinite_ids)


def part2(lines, manhattan_limit=10000):
    coords = set()
    max_x = max_y = 0

    for line in lines:
        x, y = map(int, line.split(", "))
        coords.add((x, y))
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    size_shared_region = 0

    for i in range(max_x + 1):
        for j in range(max_y + 1):
            size_shared_region += int(sum(abs(x - i) + abs(y - j) for x, y in coords) < manhattan_limit)

    return size_shared_region


lines = [line.strip() for line in open("input6.txt", "r").readlines()]
print(part1(lines))
print(part2(lines))

3290
45602


### Day 7

In [89]:
#%%writefile oj-day7.py
import re
data = [line.strip() for line in open("input7.txt", "r").readlines()]

# just for printing our list
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

## Part 1
def part_one():
    
    # grab our letters
    letters = []
    for d in data:
        letters.append([i for i in re.findall("Step ([\w]) must be finished before step ([\w]) can begin", d)][0])
    starts = [s[0] for s in letters]
    ends = [s[1] for s in letters]
    all = set(starts + ends)

    # make our order
    order = []
    while len(order) < 26:
        starts = [s[0] for s in letters]
        ends = [s[1] for s in letters]
        not_end = [a for a in all if a not in set(ends)]
        order.append(sorted(not_end)[0])
        all.remove(order[-1])
        letters = [l for l in letters if l[0] not in order]

    print(concatenate_list_data(order))
    return(order)

## Part 2

import numpy as np

def can_build(letter, letters, new_order):
    depends = set([s[0] for s in letters if s[1] == letter])
    if depends.issubset(set(new_order)):
        return True
    else:
        return False

def give_job(letter, workers, t, numbered, next_event):
    i = 0
    assigned = False
    while not assigned:
        if workers[i][0] == "-":
            workers[i][0] = letter
            workers[i][1] = t + 59 + numbered[letter]
            assigned = True
        else:
            i += 1
            
def clear_worker(workers, next_event, new_order):
    cleared = False
    for i in range(5):
        if workers[i][1] == next_event and workers[i][1] != float('Inf'):
            new_order.append(workers[i][0])
            workers[i][0] = "-"
            workers[i][1] = float('Inf')
            cleared = True
    if cleared:
        next_event += 1
    else:
        next_event = float('Inf')
        for i in range(5):
            if type(workers[i][1]) == int:
                if workers[i][1] < next_event:
                    next_event = workers[i][1]
    return(next_event)

def part_two(order):
    
    numbered = {a: i for i, a in enumerate(sorted(order), 1)}
    new_order = []
    workers = {a: ["-", float('Inf')] for a in range(5)}

    letters = []
    for d in data:
        letters.append([i for i in re.findall("Step ([\w]) must be finished before step ([\w]) can begin", d)][0])
    starts = [s[0] for s in letters]
    ends = [s[1] for s in letters]

    t = 0
    next_event = float('Inf')

    while len(new_order) < 26:  
        to_clear = []
        for o in order:
            if np.any([True for value, t in workers.values() if value == "-"]):
                if can_build(o, letters, new_order):
                    give_job(o, workers, t, numbered, next_event)
                    to_clear.append(o)

        next_event = min([w[1] for w in workers.values()])
        order = [o for o in order if o not in to_clear]
        next_event = clear_worker(workers, next_event, new_order)
        t = next_event

    print(t)       
 
order = part_one()
part_two(order)

BHMOTUFLCPQKWINZVRXAJDSYEG
877


### Day 8

In [93]:
# %%writefile oj-day8.py
import numpy as np

data = np.loadtxt('input8.txt', dtype=int)

def part_one(data):
    children, metas = data[:2]
    data = data[2:]
    scores = []
    totals = 0

    for i in range(children):
        total, score, data, scoring = part_one(data)
        totals += total
        scores.append(score)

    totals += sum(data[:metas])

    if children == 0:
        return (totals, sum(data[:metas]), data[metas:], scores)
    else:
        return (
            totals,
            sum(scores[k - 1] for k in data[:metas] if k > 0 and k <= len(scores)),
            data[metas:],
            scores
        )

total, value, remaining, scores = part_one(data)

print(total)
print(value)

45868
19724


### Day 9

In [94]:
# %%writefile oj-day9.py
from collections import deque
import re

def part_one(num_players, max_number):
    scores = {}
    game = deque([0])

    for i in range(1, max_number + 1):
        player = i % num_players
        if i % 23 == 0:
            game.rotate(7)
            if player in scores.keys():
                scores[player] += i + game.pop()
            else:
                scores[player] = i + game.pop()
            game.rotate(-1)
        else:
            game.rotate(-1)
            game.append(i)

    return max(scores.values())

data = [line.strip() for line in open("input9.txt", "r").readlines()]
print(part_one(int(re.findall("[\d]+", data[0])[0]),
          int(re.findall("[\d]+", data[0])[1])))
print(part_one(int(re.findall("[\d]+", data[0])[0]),
          int(re.findall("[\d]+", data[0])[1])*100))

384475
3187566597


### Day 10

In [95]:
# %%writefile oj-day10.py
import re
import numpy as np
data = [line.strip() for line in open("input10.txt", "r").readlines()]

coords = []
min_x = min_y = max_x = max_y = 0
for i, d in enumerate(data):
    coords.append([int(r) for r in re.findall(".[\d]+", d)])
 
for i in range(20000):
    cs = [(c[0] + (i*c[2]), c[1] + i*c[3]) for c in coords]
    min_x = min([x[0] for x in cs])
    max_x = max([x[0] for x in cs])
    min_y = min([x[1] for x in cs])
    max_y = max([x[1] for x in cs])
    height = max_y - min_y + 1
    width = max_x - min_x + 1
    if(height) < 30:
        cs = [(c[0] - min_x, c[1]) for c in cs]
        cs = [(c[0], c[1] - min_y) for c in cs]
        grid = np.chararray((height,width), unicode = True)
        grid[:] = "."
        for c in cs: grid[c[1],c[0]] = "x"
        for l in grid: print("".join(l))
        print(i)
        

...........xx......x..........................x...x.....................
....xxx...x.................xx........x.x........................x......
.......xx.............x.....x.....x......xx......x..x.x.............x...
..........x.x.x..xx.....xx.......x....xxx.x...x....x.x....x...x.........
........x.xxxx...x..x.x..x.x.....x.......x....xx....x....x.xxx....xxx...
...............x.x.x......x...x...x..x..xx.................x.....x.x...x
...xxx..xxx.x.x..xx....x.x.xxxxxx..xxx...........xx.................x...
x...x..xx...x.x.x....xx..xxx..x..xx.xx..x.x...x....x...x.x..x...........
..xx.x.xx...x..x.x...xxx.xxxx..x...x.xxx...xxx....xx.x...x..x.xxx.x...x.
....x..x...xx..xx.x.xx....x.x.xx.x.....x.x..x..x....xx.xxx....xx........
..x.x...x.xx.......x.x....x.xx.....x..xxx..x.x.x.x....xx.x..x......x.x..
......xxxx......x..x...x..x.....x.x.....x...x.x..x...xx......xx.........
..x.xx...x.xx...xx...x....x......x...xxx.x.........xx.x..x...xx..x......
...xx....xx.x.........x........x............x......

### Day 11

In [53]:
#writefile oj-day11.py
import re
import numpy as np
data = int(np.loadtxt("input11.txt"))

def power_level(coord, data):
    
    #Find the fuel cell's rack ID, which is its X coordinate plus 10.
    rack_ID = coord[0] + 10

    #Begin with a power level of the rack ID times the Y coordinate.
    power = rack_ID*coord[1]

    #Increase the power level by the value of the grid serial number (your puzzle input).
    power += data

    #Set the power level to itself multiplied by the rack ID.
    power *= rack_ID

    #Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
    power = int(str(power)[-3]) if power > 99 else 0

    #Subtract 5 from the power level.
    power -= 5
    
    return(power)

# part one
grid = np.zeros((300,300))
for i in range(1,300):
    for j in range(1,300):
        grid[i-1,j-1] = power_level((i,j),data)

grid_sums = np.zeros((298,298))        
max = 0
for i in range(298):
    for j in range(298):
        grid_sums[i,j] = np.sum(grid[i:i+3,j:j+3])
        
print((int(np.argmax(grid_sums) / 298)+1,np.argmax(grid_sums)%298 + 1))     

# part two
maxes = {}
running_max = 0
running_max_k = 0
for k in range(1,30):
    grid_sums = np.zeros((300-k+1,300-k+1))        
    max_k = 0
    for i in range(300-k+1):
        for j in range(300-k+1):
            grid_sums[i,j] = np.sum(grid[i:i+k,j:j+k])
    maxes[k] = (int(np.argmax(grid_sums) / (300-k+1)) + 1,
                int(np.argmax(grid_sums) % (300-k+1)) + 1,
                np.max(grid_sums))  
    if maxes[k][2] > running_max:
        running_max = maxes[k][2]
        running_max_k = k
        
print((maxes[running_max_k][0],maxes[running_max_k][1],running_max_k))

(21, 93)
(231, 108, 14)


### Day 12

In [148]:
#%%writefile oj-day12.py
import re
import numpy as np
data = [line.strip() for line in open("input12.txt", "r").readlines()]

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

def pot_generate(generations):
    initial = str(re.findall('initial state: (.*)',data[0])[0])
    pots = []
    for r in initial:
        pots.append(r)

    # what's the numbering of the pots
    pot_numbers = list(range(0,len(pots)))    
    first_real_pot = pot_numbers[0]
    last_real_pot = pot_numbers[-1]

    # now extend these by two * generations as we are going to add some dummy pots they fill into
    dummies = generations*2
    pot_numbers = list(range(first_real_pot-dummies,last_real_pot+dummies)) 
    pots = ['.']*dummies + pots + ['.']*dummies


    plant_rules = {}
    for d in data[2:]:
        plant_rules[d[:5]] = d[-1] 

    # pot iterate
    new_pots = pots[:]
    for i in range(1,generations+1):
        for p in range(2, len(pots)-2):
            new_pots[p] = plant_rules[concatenate_list_data(pots[p-2:p+3])]
        pots = new_pots[:]


    # add up
    val = 0
    for i, p in enumerate(pots[2:(len(pots)-2)], 2):
        if p == '#':
            val += pot_numbers[i]

    return(val)

print("part one: ", pot_generate(20))
print("part two: ", pot_generate(200) + ((50000000000 - 200) * (pot_generate(201)-pot_generate(200))))

part one:  3738
part two:  3900000002467


In [392]:
#%%writefile oj-day13.py
import re
import numpy as np
from copy import deepcopy

cart_types = ["<",">","^","v"]
cart_changes = {
    "/^":">",
    "/>":"^",
    "/<":"v",
    "/v":"<",
    "\\^":"<",
    "\\>":"v",
    "\\<":"^",
    "\\v":">",
}
intersection_changes = {
    "0^":"<",
    "0<":"v",
    "0v":">",
    "0>":"^",
    "1^":"^",
    "1<":"<",
    "1v":"v",
    "1>":">",
    "2^":">",
    "2<":"^",
    "2v":"<",
    "2>":"v",
}
turn_orders = {
    "0":"1",
    "1":"2",
    "2":"0"
}

# one copy of the map where we have removed carts
original = [line for line in open("input13.txt", "r").readlines()]
for i, l in enumerate(original):
    copy = list(l)
    for j, c in enumerate(l): 
        if c in cart_types:
            if c in ["<",">"]:
                copy[j] = "-"
            else:
                copy[j] = "|"
    original[i] = "".join(copy)

def move_cart(x, y, cn, remove_crash = False):

    # which cart is this
    cart_num = -1
    if cn == 0:
        carts[cn] = [x,y,"0"]
        cart_num = 0
        cn += 1
    else:
        for key, value in carts.items():
            if value[0] == x and value[1] == y:
                cart_num = key
        if cart_num == -1:
            cn += 1
            cart_num = cn 
            carts[cart_num] = [x,y,"0"]

    # move our cart
    new_x = x
    new_y = y

    crash = False

    if live_data[x][y] == "<":
        if live_data[x][y-1] in cart_types:
            if not remove_crash:
                print(y-1,x)
            crash = True
        live_data[x][y-1] = "<"
        new_y -= 1
    if live_data[x][y] == "^":
        if live_data[x-1][y] in cart_types:
            if not remove_crash:
                print(y,x-1)
            crash = True
        live_data[x-1][y] = "^"
        new_x -= 1
    if live_data[x][y] == ">":
        if live_data[x][y+1] in cart_types:
            if not remove_crash:
                print(y+1,x)
            crash = True
        live_data[x][y+1] = ">"
        new_y += 1
    if live_data[x][y] == "v":
        if live_data[x+1][y] in cart_types:
            if not remove_crash:
                print(y,x+1)
            crash = True
        live_data[x+1][y] = "v"
        new_x += 1

    # reset our track
    live_data[x][y] = original[x][y]
    
    # is the new position a change or intersection
    track = original[new_x][new_y]
    if track in ["/","\\"]:
        live_data[new_x][new_y] = cart_changes[original[new_x][new_y]+live_data[new_x][new_y]]
    if track == "+":
        live_data[new_x][new_y] = intersection_changes[carts[cart_num][2]+live_data[new_x][new_y]]
        carts[cart_num][2] = turn_orders[carts[cart_num][2]]

    # update our carts dict
    carts[cart_num][0] = new_x
    carts[cart_num][1] = new_y
    
    # remove carts
    if crash and remove_crash:
        return(remove_crashes(new_x,new_y))
    else:
        return ({"cn":cn,
               "crash":crash})

def process_line(line, ln, cn, remove_crash=False):
    ticker = True
    no_crash = True
    while ticker:
        for i, l in enumerate(line):
                if line[i] in cart_types:
                    res = move_cart(ln, i, cn, remove_crash)
                    cn = res["cn"]
                    if res["crash"]:
                        no_crash = False
        ticker = False
    return({"cn":cn,
           "crash": not no_crash})

def process_carts(carts, remove_crash=False):
    carts = dict(sorted(carts.items(), key=lambda x: x[1][0:2]))
    crashes = []
    for k, v in carts.items():
        if k not in crashes:
            crashes.extend(move_cart(v[0], v[1], cn, remove_crash))

def remove_crashes(x,y):
    crashes = []
    for k, v in carts.items():
        if v[0]==x and v[1]==y:
            crashes.append((k))
    for c in crashes:
        carts.pop(c)
    live_data[x][y] = original[x][y]
    return(crashes)
   
# part 1
data = [list(line) for line in open("input13.txt", "r").readlines()] # don't strip so that each line lines up right
live_data = [list(line) for line in open("input13.txt", "r").readlines()] # copy of it so that the carts don't overwrite
carts = {}
cn = 0 

no_crash = True
counter = 0
while no_crash:
    for ln, line in enumerate(data):
        res = process_line(line, ln, cn)
        cn = res["cn"]
        if res["crash"]:
            no_crash = False
    counter += 1
    data = deepcopy(live_data)
    
# part 2

data = [list(line) for line in open("input13.txt", "r").readlines()] # don't strip so that each line lines up right
live_data = [list(line) for line in open("input13.txt", "r").readlines()] # copy of it so that the carts don't overwrite
carts = {}
cn = 0

catch = 0
many_carts = True
while many_carts:
    if catch == 0:
        for ln, line in enumerate(data):
            res = process_line(line, ln, cn, remove_crash=True)
            cn = res["cn"]
        catch = 1
    else:
        process_carts(carts, True)
    data = deepcopy(live_data)
    if len(carts)==1:
        print(list(carts.values())[0][::-1][1:3])
        many_carts = False

113 136
[114, 136]


### Day 14

In [480]:
# %%writefile oj-day14.py
from collections import deque
import re

def part_one(input):
    
    recipes = [3,7]
    i = 0
    j = 1
    while len(recipes) < (input + 10):

        new = recipes[i] + recipes[j]
        if new > 9:
            recipes.extend(list(divmod(new,10)))
        else:
            recipes.append(new)
        i = ((recipes[i]+1+i) % len(recipes))
        j = ((recipes[j]+1+j) % len(recipes))
    
    print(concatenate_list_data([recipes[i] for i in range(input,input+10)]))
    
    
def part_two(input):
    
    recipes = [3,7]
    i = 0
    j = 1
    l = len(str(input))
    match = [int(s) for s in str(input)]
    while recipes[len(recipes)-l : len(recipes)] != match:

        new = recipes[i] + recipes[j]
        if new > 9:
            new = list(divmod(new,10))
            recipes.append(new[0])
            if recipes[len(recipes)-l : len(recipes)] != match:
                recipes.append(new[1])
        else:
            recipes.append(new)
        i = ((recipes[i]+1+i) % len(recipes))
        j = ((recipes[j]+1+j) % len(recipes))
        
    print(len(recipes) - l)
    
data = int(np.loadtxt("input14.txt"))
part_one(data)
part_two(data)

1611732174
20279772


### Day 15:

In [481]:
# %%writefile oj-day15.py
import sys
from collections import deque
from dataclasses import dataclass

def sim(elf_attack=3, loud = False):
    
    gob_attack = 3

    @dataclass(order=True)
    class Unit:
        y : int
        x : int
        hp : int
        typ : str
        @property
        def attack(self):
            return elf_attack if self.typ == 'E' else gob_attack
        def dist(a,b):
            return abs(a.x-b.x) + abs(a.y-b.y)
        def __str__(self):
            return f'[{self.typ}: {self.hp}]'

    G = []
    E = []
    M = []
    for i,line in enumerate([line for line in open("input15.txt", "r").readlines()]):
        line = line.strip()
        for j,c in enumerate(line):
            if c == 'G':
                G.append(Unit(i,j,200,'G'))
            if c == 'E':
                E.append(Unit(i,j,200,'E'))
        M.append([int(c == '#') for c in line])

    elfded = False

    def viz():
        H = len(M)
        W = len(M[0])
        Mc = [m[:] for m in M]

        units = sorted(G + E)

        for e in units:
            if e.typ == 'E': Mc[e.y][e.x] = 3
            else:            Mc[e.y][e.x] = 2

        by_row = [[] for _ in range(H)]
        for f in sorted(G + E):
            by_row[f.y].append(f'[{f.typ}: {f.hp}]')

        for i in range(H):
            s = ''.join('.#GE'[Mc[i][j]] for j in range(W))
            print(s, ' '.join(by_row[i]))
        print()

    def neigh(x,y):
        return [(x,y-1),(x-1,y),(x+1,y),(x,y+1)]

    turns = 0
    while E and G:
        units = sorted(G+E)

        for unit in units:
            if unit.typ == 'X': continue

            enemies = sorted(e for e in units if e.typ != unit.typ)
            targets = set().union(*(set(neigh(e.x,e.y)) for e in enemies if e.typ != 'X'))

            cands = []
            goald = 1e9
            blocked = {(e.x,e.y) for e in units if e.typ != 'X'}
            Q = deque([(unit.x,unit.y,0,None)])
            while Q:
                x,y,d,step1 = Q.popleft()
                if d > goald: break
                if M[y][x] or ((x,y) in blocked and d > 0): continue
                blocked.add((x,y))

                if d == 1:
                    step1 = (x,y)
                if (x,y) in targets:
                    goald = d
                    cands.append([y,x,d,step1])
                for s,t in neigh(x,y):
                    Q.append((s,t,d+1,step1))

            if len(cands) == 0: continue

            x,y,d,step1 = min(cands)
            if d > 0:
                unit.x,unit.y = step1
            if d <= 1:
                target = min((e for e in enemies if e.typ != 'X' and e.dist(unit) == 1),
                             key=lambda x: x.hp)
                target.hp -= unit.attack
                if target.hp <= 0:
                    if target.typ == 'E':
                        elfded = True
                    target.typ = 'X'

        # Purge the dead
        E = [e for e in E if e.typ != 'X']
        G = [e for e in G if e.typ != 'X']

        turns += 1
        #print(f'=== round {turns} ===')
        #viz()

    hp = sum(e.hp for e in G) + sum(e.hp for e in E)
    if loud:
        print((turns-1)*hp)
    return([(turns-1)*hp,elfded])

# part one
sim(3, loud = True)

# part two
died = True
elf_power = 4
while died:
    out = sim(elf_power, loud = False)
    if not out[1]:
        died = False
        print(out[0])
    elf_power += 1

228730
33621
